Import Inception v_3 

In [31]:
from keras.applications.inception_v3 import InceptionV3

import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import image
from keras import utils, Model, layers, optimizers, losses
from keras.models import Sequential, load_model
from keras.layers import Conv2D, Flatten, MaxPooling2D, Dense, Dropout, GlobalAveragePooling2D
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers.legacy import RMSprop
from keras.preprocessing.image import ImageDataGenerator


In [32]:
# load the weights
weights_file = '/Users/samin/Desktop/ML-Projects/tensorflow_practice/transfer learning/inception_v3_weights_tf_dim_ordering_tf_kernels_notop.h5'

# create the inception model with the weights
pre_trained_model = InceptionV3(input_shape=(150, 150, 3),
                        include_top=False,
                        weights=weights_file)

In [33]:
# All images will be rescaled by 1./255
training_dir = '/Users/samin/Desktop/ML-Projects/tensorflow_practice/data/horse-or-human/train'
train_datagen = ImageDataGenerator(rescale=1/255)
train_generator = train_datagen.flow_from_directory(
                    training_dir,
                    target_size=(150, 150),
                    class_mode = 'binary'
)

val_dir = '/Users/samin/Desktop/ML-Projects/tensorflow_practice/data/horse-or-human/validation'
val_datagen = ImageDataGenerator(rescale=1/255)
val_generator = val_datagen.flow_from_directory(
                    val_dir,
                    target_size=(150, 150),
                    class_mode = 'binary'
)

Found 1027 images belonging to 2 classes.
Found 256 images belonging to 2 classes.


In [34]:
# pre_trained_model.summary()

In [35]:
# freeze the pre_trained_model
for layer in pre_trained_model.layers:
   layer.trainable = False

# stop our model at 'mixed7' layer
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

# to use the last layer of a model, try model.layers[-1]

last layer output shape:  (None, 7, 7, 768)


In [36]:
# flatthen our last layer and add two more dense layers
x = layers.Flatten()(last_output)
x = layers.Dense(units=1024, activation='relu')(x)
x = layers.Dense(units=1, activation='sigmoid')(x)

In [37]:
model = Model(pre_trained_model.input, x)

model.compile(optimizer=RMSprop(lr=0.001), loss='binary_crossentropy',
                metrics=['accuracy'])

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/keras/src/optimizers/legacy/rmsprop.py:144: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [38]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, logs={}):
        if logs.get('accuracy')>0.99:
            print("\nReached 99% test accuracy so cancelling training!")
            self.model.stop_training = True

    # def on_epoch_end_val(self, logs={}):
    #     if logs['val_accuracy']>=0.99:
    #         print("\nReached 99% validation accuracy so cancelling training!")
    #         self.model.stop_training = True
    
callback = myCallback()

# Create the EarlyStopping callback
# class CustomCallBack(EarlyStopping):
#     def on_end(self, logs=None):
#         if self.stopped_epoch > 0:
#             print(f"\nTraining stopped after {self.stopped_epoch} epochs due to early stopping.")
#         else:
#             print("Training completed without early stopping.")

# early_stopping_callback = CustomCallBack(monitor='val_accuracy', patience=3, mode='max', verbose=1)
early_stopping_callback = EarlyStopping(monitor='val_accuracy', patience=3, mode='max', verbose=1)

In [39]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_4 (InputLayer)        [(None, 150, 150, 3)]        0         []                            
                                                                                                  
 conv2d_282 (Conv2D)         (None, 74, 74, 32)           864       ['input_4[0][0]']             
                                                                                                  
 batch_normalization_282 (B  (None, 74, 74, 32)           96        ['conv2d_282[0][0]']          
 atchNormalization)                                                                               
                                                                                                  
 activation_282 (Activation  (None, 74, 74, 32)           0         ['batch_normalization_28

In [40]:
history = model.fit(train_generator, validation_data=val_generator, 
                    callbacks=early_stopping_callback, epochs=40, verbose=1)

Epoch 1/40
33/33 [==============================] - 17s 460ms/step - loss: 0.2787 - accuracy: 0.9698 - val_loss: 5.4158e-04 - val_accuracy: 1.0000
Epoch 2/40
33/33 [==============================] - 15s 457ms/step - loss: 0.0025 - accuracy: 0.9990 - val_loss: 8.4473 - val_accuracy: 0.5117
Epoch 3/40
33/33 [==============================] - 14s 420ms/step - loss: 0.3402 - accuracy: 0.9766 - val_loss: 1.8578e-13 - val_accuracy: 1.0000
Epoch 4/40
33/33 [==============================] - 15s 443ms/step - loss: 1.0583e-08 - accuracy: 1.0000 - val_loss: 1.7929e-13 - val_accuracy: 1.0000
Epoch 4: early stopping
